Its very important to know the type of Data to decide which Machine learning algorthim you have to use.There are three types of Data:

* Categorical Data
* Numerical Data
* Ordinal Data

In these notebook we will perfom following task:
* First we will look through data and explore
* We will see if data contains any missing value and try to fix it.
* Insights of data are always usefull hence we will explore through seaborn , matplotlib libraries.
* We will deploy through Logistic Regression Model.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/loan-predication'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
train = pd.read_csv('/kaggle/input/loan-predication/train_u6lujuX_CVtuZ9i (1).csv')
test = pd.read_csv('../input/av-test/test.csv')

In [ ]:
train.shape

In [ ]:
test.head()

In [ ]:
train.isnull().sum()

In [ ]:
train.info()

**First we deal with the missing values** 

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(10,8)})
sns.countplot('Loan_Status',hue='Gender',data=train
             )

In [ ]:
train['self'] = 'train'
test['self'] = 'test'
raw_data = pd.concat([train,test])

*Most probably after looking through the graph we are going to fill with mode of the data i.e 'Male'*

In [ ]:
raw_data['Gender'] = raw_data['Gender'].fillna('Male')

In [ ]:
sns.countplot('Loan_Status',hue='Married',data=raw_data)

We can infer from this that those who were married were most probably to get the loan.

In [ ]:
raw_data['Married'] = raw_data['Married'].fillna('Yes')

In [ ]:
raw_data['Dependents'] = raw_data['Dependents'].fillna(-99)
for i in range(len(raw_data)):
    if(raw_data['CoapplicantIncome'].iloc[i]!=0 and raw_data['Dependents'].iloc[i]==-99 ):
        raw_data['Dependents'].iloc[i]= '1'
    elif(raw_data['CoapplicantIncome'].iloc[i]==0 and raw_data['Dependents'].iloc[i]==-99 ):
        raw_data['Dependents'].iloc[i]=='0'
    else:
        continue
        
        


In [ ]:
raw_data.isnull().sum()

In [ ]:
raw_data['more_than_one_dependents'] = raw_data['Dependents'].apply(lambda x : 1 if x!='0' else 0)

In [ ]:
raw_data['no_dependent'] = raw_data['Dependents'].apply(lambda x : 1 if x=='0' else 0)

In [ ]:
raw_data.head()

Since most of the dependents are 0 we are gonna fill with it with 0

In [ ]:
sns.countplot('Loan_Status',hue='Self_Employed',data=raw_data)

In [ ]:
raw_data['Self_Employed'] = raw_data['Self_Employed'].fillna('No')

In [ ]:
sns.distplot(raw_data['LoanAmount'])

In [ ]:
sns.scatterplot(raw_data['LoanAmount'],y=np.arange(0,981),hue=raw_data['Loan_Status'])

We see outliers as data seems to right skewed , so we will not fill values by mean of this data, we gonna take the mean without containing outliers.

In [ ]:
mean=raw_data[raw_data['LoanAmount']<=200]['LoanAmount'].mean()
raw_data['LoanAmount']=raw_data['LoanAmount'].fillna(mean)


In [ ]:
sns.scatterplot(raw_data['Loan_Amount_Term'],y=np.arange(0,981),hue=raw_data['Loan_Status'])

For the missing value we are going to use mode.

In [ ]:
raw_data['Loan_Amount_Term'].fillna(raw_data['Loan_Amount_Term'].value_counts().idxmax(), inplace=True)


In [ ]:
raw_data['Credit_History'].unique()

In [ ]:
sns.countplot('Loan_Status',hue='Credit_History',data=raw_data)

In [ ]:
sns.scatterplot(raw_data['Credit_History'],y=np.arange(0,981),hue=raw_data['Loan_Status'])

*Again we are going to use mode*

In [ ]:
raw_data['Credit_History'].fillna(raw_data['Credit_History'].value_counts().idxmax(), inplace=True)



In [ ]:
raw_data.isnull().sum()

In [ ]:
raw_data.head()


In [ ]:
raw_data['total_income'] = raw_data['ApplicantIncome'] + raw_data['CoapplicantIncome']
raw_data['EMI'] = (raw_data['LoanAmount']*0.09*(1.09**raw_data['Loan_Amount_Term']))/(1.09**(raw_data['Loan_Amount_Term']-1))


In [ ]:
raw_data['Loan_Amount_Term'].value_counts()
raw_data['common_loan_term'] = raw_data['Loan_Amount_Term'].apply(lambda x : 1 if x==360.0 else 0)
raw_data['uncommon_loan_term'] = raw_data['Loan_Amount_Term'].apply(lambda x: 1 if x!=360.0 else 0)


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,12))
sns.heatmap(raw_data.corr(),annot=True,cbar=False)

In [ ]:
raw_data = raw_data.drop(['Loan_ID','ApplicantIncome','CoapplicantIncome','Dependents','Loan_Amount_Term'],1)

# Part 2
We are first going to one hot encode every Categorcial data then we are going to deploy it in our model

In [ ]:
train = raw_data[raw_data['self']=='train']
train = train.drop(['self'],1)
test = raw_data[raw_data['self']=='test']
test = test.drop(['self','Loan_Status'],1)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
from sklearn.metrics import roc_auc_score

Let us check if the category doesnt contain any duplicates such as yes, YES ...

In [ ]:
print(train['Gender'].unique())
print(train['Married'].unique())
print(train['Education'].unique())
print(train['Self_Employed'].unique())
print(train['Property_Area'].unique())

In [ ]:
train['Gender'] = lb.fit_transform(train['Gender'])
train['Married'] = lb.fit_transform(train['Married'])
train['Education'] = lb.fit_transform(train['Education'])
train['Self_Employed'] = lb.fit_transform(train['Self_Employed'])
train['Property_Area'] = lb.fit_transform(train['Property_Area'])
train['Loan_Status'] = lb.fit_transform(train['Loan_Status'])


test['Gender'] = lb.fit_transform(test['Gender'])
test['Married'] = lb.fit_transform(test['Married'])
test['Education'] = lb.fit_transform(test['Education'])
test['Self_Employed'] = lb.fit_transform(test['Self_Employed'])
test['Property_Area'] = lb.fit_transform(test['Property_Area'])



Lets see our result

In [ ]:
print(train['Gender'].unique())
print(train['Married'].unique())
print(train['Education'].unique())
print(train['Self_Employed'].unique())
print(train['Property_Area'].unique())
print(train['Loan_Status'].unique())


# Part 3
Deploying it in the model

In [ ]:
from sklearn.preprocessing import StandardScaler,RobustScaler
from sklearn.model_selection import train_test_split
rb = RobustScaler()

In [ ]:
test.head()

In [ ]:
col = ['LoanAmount','total_income','EMI']
for i in col:
    train[[i]] = rb.fit_transform(train[[i]])
    test[[i]] = rb.transform(test[[i]])

In [ ]:
X = train.drop(['Loan_Status'],axis=1)
y = train['Loan_Status']



In [ ]:
X.describe()

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

In [ ]:
def training(model,X,y):
    from sklearn.model_selection import train_test_split
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size =0.25,random_state=42,stratify=y)
    model.fit(X_train,y_train)
    preds_train = model.predict(X_train)
    preds_test = model.predict(X_test)
    print("ROC score for train:",roc_auc_score(y_train,preds_train))
    print("ROC score for test:",roc_auc_score(y_test,preds_test))
    print('\n')
    print("Confusion Matrix for train :\n",confusion_matrix(y_train,preds_train))
    print("Confusion Matrix for test :\n",confusion_matrix(y_test,preds_test))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix
lgt = LogisticRegression()
training(lgt,X,y)

In [ ]:
lgt_preds = lgt.predict(test)
sub = pd.read_csv('../input/sample-sub/sample_submission_49d68Cx.csv')
sub['Loan_Status'] = lgt_preds
sub['Loan_Status'] = sub['Loan_Status'].replace({1:'Y',0:'N'})
sub.to_csv('lgt_prediction_2.csv',index=False)

**Hence from Logistic regression we get an accuracy of about 85%**<br>
Just as starting our Logisitic Regression performed well.

LightGBM

In [ ]:
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier

oof_pred               = np.zeros((len(train),))
y_pred_final           = np.zeros((len(test),))
num_models             = 3

n_splits               = 20
error                  = []

kf=StratifiedKFold(n_splits=n_splits,shuffle=True,random_state=294)
    
for i,(train_idx,val_idx) in enumerate(kf.split(X,y)):    
    
    wghts                     = [0]*num_models
    test_roc_score            = []
    
    
    X_train, y_train = X.iloc[train_idx,:], y.iloc[train_idx]

    X_val, y_val = X.iloc[val_idx, :], y.iloc[val_idx]
    

    print('\nFold: {}\n'.format(i+1))

    model1 = LGBMClassifier(boosting_type='gbdt',n_estimators=500,depth=-1,learning_rate=0.03,scale_pos_weight=7,objective='binary',metric='auc',
                 colsample_bytree=0.5,random_state=294,n_jobs=-1)
    model1.fit(X_train,y_train)
    testpred1 = model1.predict_proba(X_val)[:,1]
    test_roc_score.append(roc_auc_score(y_val, testpred1))
    print("Test ROC AUC for model 1: %.4f"%(roc_auc_score(y_val, testpred1)))
    
    model2 = LGBMClassifier(boosting_type='gbdt',n_estimators=300,depth=-1,learning_rate=0.03,scale_pos_weight=7,objective='binary',metric='auc',
                 colsample_bytree=0.3,reg_alpha=2,random_state=294,n_jobs=-1)
    model2.fit(X_train,y_train)
    testpred2 = model2.predict_proba(X_val)[:,1]
    test_roc_score.append(roc_auc_score(y_val, testpred2))
    print("Test ROC AUC for model 2: %.4f"%(roc_auc_score(y_val, testpred2)))
    
    model3 = LGBMClassifier(boosting_type='gbdt',n_estimators=400,depth=-1,learning_rate=0.03,scale_pos_weight=7,objective='binary',metric='auc',
                 colsample_bytree=0.4,reg_alpha=2,random_state=294,n_jobs=-1)
    model3.fit(X_train,y_train)
    testpred3 = model3.predict_proba(X_val)[:,1]
    test_roc_score.append(roc_auc_score(y_val, testpred3))
    print("Test ROC AUC for model 3: %.4f"%(roc_auc_score(y_val, testpred3)))
    
    wghts              = np.exp(-1000*np.array(test_roc_score/sum(test_roc_score)))
    wghts              = wghts/sum(wghts)
    
    val_pred           = wghts[0]*testpred1+wghts[1]*testpred2 +wghts[2]*testpred3
    print('validation roc_auc_score fold-',i+1,': ',roc_auc_score(y_val, val_pred))
    
    oof_pred[val_idx]  = val_pred
    y_pred_final += (wghts[0]*model1.predict_proba(test)[:,1]+wghts[1]*model2.predict_proba(test)[:,1]+wghts[2]*model3.predict_proba(test)[:,1])/(n_splits)
    
    print('\n')
    
print('OOF ROC_AUC_Score:- ',(roc_auc_score(y,oof_pred)))

CATBOOST

In [ ]:
from catboost import CatBoostClassifier
oof_pred               = np.zeros((len(train),))
y_pred_final           = np.zeros((len(test),))
num_models             = 3

n_splits               = 20
error                  = []

kf=StratifiedKFold(n_splits=n_splits,shuffle=True,random_state=294)
    
for i,(train_idx,val_idx) in enumerate(kf.split(X,y)):    
    
    wghts                     = [0]*num_models
    test_roc_score            = []
    
    
    X_train, y_train = X.iloc[train_idx,:], y.iloc[train_idx]

    X_val, y_val = X.iloc[val_idx, :], y.iloc[val_idx]
    

    print('\nFold: {}\n'.format(i+1))

    model1 = CatBoostClassifier(learning_rate = 0.03,random_state=42,scale_pos_weight=7, custom_metric=['AUC'])
    model1.fit(X_train,y_train,eval_set=(X_val, y_val),early_stopping_rounds=30,verbose=100)
    testpred1 = model1.predict_proba(X_val)[:,1]
    test_roc_score.append(roc_auc_score(y_val, testpred1))
    print("Test ROC AUC for model 1: %.4f"%(roc_auc_score(y_val, testpred1)))
    
    model2 = CatBoostClassifier(learning_rate = 0.04,random_state=42,scale_pos_weight=7, custom_metric=['AUC'])
    model2.fit(X_train,y_train,eval_set=(X_val, y_val),early_stopping_rounds=40,verbose=100)
    testpred2 = model2.predict_proba(X_val)[:,1]
    test_roc_score.append(roc_auc_score(y_val, testpred2))
    print("Test ROC AUC for model 2: %.4f"%(roc_auc_score(y_val, testpred2)))
    
    model3 = CatBoostClassifier(learning_rate = 0.05,random_state=42,scale_pos_weight=7, custom_metric=['AUC'])
    model3.fit(X_train,y_train,eval_set=(X_val, y_val),early_stopping_rounds=20,verbose=100)
    testpred3 = model3.predict_proba(X_val)[:,1]
    test_roc_score.append(roc_auc_score(y_val, testpred3))
    print("Test ROC AUC for model 3: %.4f"%(roc_auc_score(y_val, testpred3)))
    
    wghts              = np.exp(-1000*np.array(test_roc_score/sum(test_roc_score)))
    wghts              = wghts/sum(wghts)
    
    val_pred           = wghts[0]*testpred1+wghts[1]*testpred2 +wghts[2]*testpred3
    print('validation roc_auc_score fold-',i+1,': ',roc_auc_score(y_val, val_pred))
    
    oof_pred[val_idx]  = val_pred
    y_pred_final += (wghts[0]*model1.predict_proba(test)[:,1]+wghts[1]*model2.predict_proba(test)[:,1]+wghts[2]*model3.predict_proba(test)[:,1])/(n_splits)
    
    print('\n')
    
print('OOF ROC_AUC_Score:- ',(roc_auc_score(y,oof_pred)))

In [ ]:
cb_preds = cb.predict(test)
sub = pd.read_csv('../input/sample-sub/sample_submission_49d68Cx.csv')
sub['Loan_Status'] = cb_preds
sub['Loan_Status'] = sub['Loan_Status'].replace({1:'Y',0:'N'})
sub.to_csv('cb_prediction_2.csv',index=False)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(base_estimator=lgt)
ada.fit(X_train,y_train)
ada.score(X_test,y_test)

Using SVM

In [ ]:
from sklearn import svm
from sklearn.metrics import accuracy_score
clf = svm.SVC(kernel='linear')
clf.fit(X_train,y_train)
preds = clf.predict(X_test)
print(accuracy_score(y_test,preds))

In [ ]:
import lightgbm as lgb
model = lgb.LGBMClassifier(objective='mse', seed=8798, num_threads=1)
model.fit(X_train, y_train, eval_set=[(X_test, y_test), (X_train, y_train)], verbose=10)
lgb.plot_metric(model)


In [ ]:
model.score(X_test,y_test)

In [ ]:
model_preds = model.predict(test)
sub = pd.read_csv('../input/sample-sub/sample_submission_49d68Cx.csv')
sub['Loan_Status'] = model_preds
sub['Loan_Status'] = sub['Loan_Status'].replace({1:'Y',0:'N'})
sub.to_csv('model_lgb_prediction.csv',index=False)

**THERE IS AN OVERFITTING**<br>
We need to tune parameters to prevent the overfitting.

**Consider this as just starting model, what more could be done:<br>**

1) Hyperparamater tunings of best model.<br>
2) Adding features using best feature engineering.<br>
3) I haven't tried neural network but you can use it.<br>
4) More over we can achieve atleast accuracy of 0.82-0.85, using proper hyperparameter tuning, and <br>
   using additional features.

# Feel free to comment any doubt , if you got more better result or thought of any other feature engineering you can comment, as sharing make us stronger.
** Thanks if you liked my work dont forget to upvote. **